# Proyecto Music Stream

## Fase 1: Extraer Datos

1. Extraer información de Api Spotify
2. Extraer información de API Last FM
3. Guardar esa información en CSV



### Spotify

In [39]:
# API Spotify = 'https://spotipy.readthedocs.io/en/2.24.0/'

In [40]:
pip install spotipy


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [41]:
import requests
import pandas as pd
import time 
import os #para manejar archivos
import csv 
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [42]:
# Credenciales:
client_ID = 'a018711755a146b2a678a93741dc6041'
client_SECRET = '51fa44e4b57343f19c810b9a75089da6'

# Autenticación:
auth_manager = SpotifyClientCredentials(client_id=client_ID, client_secret=client_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

Géneros sobre los que trabajaremos:
- Pop
- Rock
- Rap
- Reggaeton

Rango de años:
> 2018-2022

In [43]:
genres = ['pop', 'rock', 'rap', 'reggaeton']  #lista de géneros musicales que queremos buscar
year_start = 2018  #año de inicio
year_end = 2022  #año de fin
limit = 50 #50 es el numero de datos que la API nos da de resultado por consulta
all_data = []  #lista donde guardamos los datos obtenidos


In [44]:
for genre in genres:  #iteramos por genero y año para asegurarnos los resultados por separado
    for year in range(year_start, year_end + 1):
        offset = 0  #empezamos desde el primer resultado

        while True:  #usamos el bucle while para hacer más consultas y obtener mas de 50 resultados si existen, esto es la paginación 
            query = f"genre:{genre} year:{year}" #construimos la consulta en la API

            try:
                results_search = sp.search(q=query, type='track', limit=limit, offset=offset) #hacemos la llamada a la API y le pedimos 50 resultados por consulta

                if 'tracks' in results_search and results_search['tracks']['items']:  #comprobamos que haya resultados
                    for item in results_search['tracks']['items']:
                        release_date = item.get('album', {}).get('release_date', 'Unknown')
                        release_year = release_date[:4] if release_date != 'Unknown' else 'Unknown'

                        data = {  #extraemos los datos
                           'artist': item.get('artists', [{}])[0].get('name', 'Unknown'),  #artista
                           'genre': genre,  #genero
                           'album': item.get('album', {}).get('name', 'Unknown'),  # Obtener el nombre del álbum
                           'track': item.get('name', 'Unknown'),  #cancion
                           'year': release_year,  #año de lanzamiento
                        'spotify_id': item.get('id', 'Unknown')  #ID de Spotify
                        }

                        all_data.append(data)  #guardamos todos los datos
                        print(f"{data['track']} - {data['artist']} (Año: {data['year']})")
                else:
                    print(f"No hay más resultados para {genre} en {year}") 
                    break  #si no hay más resultados salimos del bucle
                offset += limit  #pasamos a la siguiente "pagina", es decir, buscamos otros 50 resultados más 
                
                if not results_search['tracks']['next']: #si no hay mas paginas salimos del bucle
                    break  

            except Exception as e:
                print(f"Error en la consulta {query}: {e}")
                break  #Si hay un error tambien salimos del bucle

print(f"\nFin de la extracción de datos. Se han obtenido: {len(all_data)} registros.")  #display en pantalla de los resultados obtenidos


Cuando Nadie Ve - Morat (Año: 2018)
Besos En Guerra - Morat (Año: 2018)
Qué Bonito Es Querer - Manuel Carrasco (Año: 2018)
No Se Va - Morat (Año: 2018)
Destino o casualidad - Melendi (Año: 2018)
Always Remember Us This Way - Lady Gaga (Año: 2018)
Shallow - Lady Gaga (Año: 2018)
lovely (with Khalid) - Billie Eilish (Año: 2018)
Perdona (Ahora Sí Que Sí) - Carolina Durante (Año: 2018)
Presiento - Morat (Año: 2018)
Báilame - Remix - Nacho (Año: 2018)
Traicionera - Sebastian Yatra (Año: 2018)
Solita - Ozuna (Año: 2018)
No Hay Nadie Más - Sebastian Yatra (Año: 2018)
Pausa - IZAL (Año: 2018)
Déjala Que Baile (with Alejandro Sanz & Arkano) - Melendi (Año: 2018)
God's Plan - Drake (Año: 2018)
Snowman - Sia (Año: 2018)
Vaina Loca - Ozuna (Año: 2018)
Felices los 4 - Maluma (Año: 2018)
Natural - Imagine Dragons (Año: 2018)
Te Vi - Piso 21 (Año: 2018)
No Lie - Sean Paul (Año: 2018)
Call Out My Name - The Weeknd (Año: 2018)
Happier - Marshmello (Año: 2018)
Let Me Down Slowly - Alec Benjamin (Año: 20

In [45]:
#Limpieza de datos obtenidos de spotify

df_spotify=pd.DataFrame (all_data)

# Quitar los duplicados
df_spotify.drop_duplicates(subset=['artist', 'track', 'year'], keep='first', inplace=True)

# Sustituir espacios vacios
df_spotify.replace('Unknown', pd.NA, inplace=True)

print(df_spotify)

               artist      genre                 album  \
0               Morat        pop        Balas Perdidas   
1               Morat        pop        Balas Perdidas   
2     Manuel Carrasco        pop      La Cruz Del Mapa   
3               Morat        pop        Balas Perdidas   
4             Melendi        pop  Destino o casualidad   
...               ...        ...                   ...   
1994      Ryan Castro  reggaeton           Reggaetonea   
1995           Blessd  reggaeton        Siempre Blessd   
1996    Justin Quiles  reggaeton                 AEIOU   
1997           Camilo  reggaeton  De Adentro Pa Afuera   
1999          Becky G  reggaeton              ESQUEMAS   

                        track  year              spotify_id  
0             Cuando Nadie Ve  2018  71iAechwVKGu26pahzFL0k  
1             Besos En Guerra  2018  1mlGScrDQqHqmhdIqE8MmA  
2        Qué Bonito Es Querer  2018  6uzGmmQ4CuG9HvZktEy0LF  
3                    No Se Va  2018  4khWEpnaijN0G1x019

In [ ]:
file_name = "spotify_data.csv" #definimos como vamos a llamar al archivo donde guardamos los datos

#creamos un nuevo nombre único si el archivo ya existe
if os.path.exists(file_name): #con este comando verificamos si el archivo que queremos crear ya existe
    i = 1  #añadimos un contador por si existe el archivo
    while os.path.exists(f"spotify_data_{i}.csv"):  #buscamos un numero libre
        i += 1
    file_name = f"spotify_data_{i}.csv"  #creamos un nuevo nombre para el archivo y lo mostramos en pantalla
    print(f"El archivo que quieres guardar ya existe, se guardara uno nuevo como: {file_name}")

df_spotify.to_csv(file_name, index=False, encoding='utf-8', sep=',')
#guardamos el archivo asegurandonos que:
#sep=',': la separación entre columnas es una coma
#encoding='utf-8': se mantengan los caracteres especiales
#index=False : evita las columnas de indices


El archivo que quieres guardar ya existe, se guardara uno nuevo como: spotify_data_1.csv


### LAST FM

In [47]:
# API LAST FM = ' http://ws.audioscrobbler.com/2.0/'

In [48]:
pip install pylast


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [49]:
import pylast

In [50]:
#Nos conectamos a LastFM

In [51]:
# Credenciales
lastfm_client_ID = '116e34dc50ec18aa836be2e0387d273d'
lastfm_client_SECRET = '439994eca716ee61771960e8e116f946'

# Autenticación
network = pylast.LastFMNetwork(api_key=lastfm_client_ID, api_secret=lastfm_client_SECRET)



In [52]:
artist_spotify = df_spotify['artist']

In [53]:
#Creamos una lista de artistas, llamada artist_spotify, a base de dataframe df_spotify

In [54]:
artist_spotify = []
for index, row in df_spotify.iterrows():
    artist_spotify.append(row['artist'])

In [55]:
print (artist_spotify)

['Morat', 'Morat', 'Manuel Carrasco', 'Morat', 'Melendi', 'Lady Gaga', 'Lady Gaga', 'Billie Eilish', 'Carolina Durante', 'Morat', 'Nacho', 'Sebastian Yatra', 'Ozuna', 'Sebastian Yatra', 'IZAL', 'Melendi', 'Drake', 'Sia', 'Ozuna', 'Maluma', 'Imagine Dragons', 'Piso 21', 'Sean Paul', 'The Weeknd', 'Marshmello', 'Alec Benjamin', 'ZAYN', 'Camila Cabello', 'Imagine Dragons', 'Charlie Puth', 'TINI', 'The Neighbourhood', 'Calvin Harris', 'Jonas Blue', 'Sebastian Yatra', 'Maka', 'The Neighbourhood', 'Becky G', 'IZAL', 'girl in red', 'The Weeknd', 'Viva Suecia', 'NATTI NATASHA', 'Sinsinati', 'Ozuna', 'Sebastian Yatra', 'Cardi B', 'Piso 21', 'Pedro Capó', 'Jonas Blue', 'Ozuna', 'Calum Scott', '5 Seconds of Summer', 'Clean Bandit', 'Ellie Goulding', 'Post Malone', 'Panic! At The Disco', 'Kany García', 'Post Malone', 'George Ezra', 'Jubël', 'Maluma', 'Morat', 'Morat', 'BTS', 'Ozuna', 'Lewis Capaldi', 'Thalia', 'Fonseca', 'Ariana Grande', 'Ariana Grande', 'Shakira', 'Ozuna', 'The Neighbourhood', 'D

Datos que extraeremos:
- Biography
- Playcount
- Listeners
- Similar artists

Rango de años:
> 2018-2022

In [56]:
# Inicializamos la lista para en un futuro almacenar toda la información de los artistas
all_data_lastfm = []

# Iterar sobre la lista de artistas obtenidos de Spotify 
for artist in artist_spotify:
    time.sleep(1)
    # Construimos la URL para obtener la información del artista en Last.fm
    url_artist_info = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist}&api_key={lastfm_client_ID}&format=json"
    
    # Realizamos la petición HTTP
    try:
        results_artists = requests.get(url_artist_info)
        results_json = results_artists.json()  # Convertimos la respuesta a formato JSON
        
        # Comprobamos si la respuesta contiene la clave 'artist' 
        if 'artist' in results_json:
            artist_data = results_json['artist']
            
            #  Primero extraemos los datos sobre la biografía, playcount y listeners
            biography = artist_data['bio'].get('summary', 'No biography available')
            playcount = artist_data['stats'].get('playcount', 'No playcount available')
            listeners = artist_data['stats'].get('listeners', 'No listeners available')
            
            # Ahora, obtenemos los datos sobre artistas similares:
            url_similars = f'http://ws.audioscrobbler.com/2.0/?method=artist.getsimilar&artist={artist}&api_key={lastfm_client_ID}&format=json'
            response_similars = requests.get(url_similars)
            
            if response_similars.status_code == 200:
                data_similars = response_similars.json()
                if 'similarartists' in data_similars:
                    similars = [similar['name'] for similar in data_similars['similarartists']['artist']]
                else:
                    similars = []  # Si no hay artistas similares
            else:
                similars = []  # Si la solicitud falla o el código de estado no es 200

            # Agregamos los datos obtenidos de cada artista a la lista
            all_data_lastfm.append({
                'Artist': artist,
                'Biography': biography,
                'Playcount,': playcount,
                'Listeners,': listeners,
                'Similars': ', '.join(similars)  # Almacenamos los artistas similares como una cadena de texto
            })

        else:
            # Si no se encuentra la clave 'artist', mostramos un mensaje y continuamos
            print(f"Datos no disponibles para el artista: {artist}")

    except requests.exceptions.RequestException as e:
        # En caso de error en la solicitud (por ejemplo, problemas de conexión)
        print(f"Error al obtener datos de Last.fm para el artista {artist}: {e}")
        continue



Error al obtener datos de Last.fm para el artista Dean Lewis: Expecting value: line 1 column 1 (char 0)
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud
Datos no disponibles para el artista: Funzo & Baby Loud


In [57]:
# Crear un DataFrame a partir de la lista 'all_data_lastfm' (con todos los artistas)
df_last_fm = pd.DataFrame(all_data_lastfm)
#Limpieza de datos obtenidos de LastFm
# Limpiar los nombres de las columnas, eliminando comillas, comas y otros caracteres invisibles
df_last_fm.columns = df_last_fm.columns.str.replace('"', '', regex=True).str.strip()  # Eliminar comillas
df_last_fm.columns = df_last_fm.columns.str.replace(',', '', regex=True).str.strip()  # Eliminar las comas
# Confirmar que las columnas están bien limpiadas
print(df_last_fm.columns)
# Eliminar duplicados por 'Artist', 'Biography', y 'Playcount'
df_last_fm.drop_duplicates(subset=['Artist', 'Biography', 'Playcount'], keep='first', inplace=True)  # Eliminar duplicados
# Reemplazar valores nulos, vacíos y 'Unknown' por NA
df_last_fm.replace([None, '', 'Unknown'], pd.NA, inplace=True)
# Limpiar la columna 'Similars' eliminando espacios y reemplazando vacíos por NA
df_last_fm['Similars'] = df_last_fm['Similars'].str.strip()  # Eliminar espacios en blanco en la columna 'Similars'
df_last_fm['Similars'].replace('', pd.NA, inplace=True)  # Reemplazar vacíos por NA
# Eliminar filas con valores nulos en 'Biography' o 'Playcount'
df_last_fm.dropna(subset=['Biography', 'Playcount'], inplace=True)
# Mostrar el DataFrame limpio
print("DataFrame limpio:")
print(df_last_fm)








Index(['Artist', 'Biography', 'Playcount', 'Listeners', 'Similars'], dtype='object')
DataFrame limpio:
               Artist                                          Biography  \
0               Morat  Morat is a Colombian band formed in the countr...   
1               Morat  Morat is a Colombian band formed in the countr...   
2     Manuel Carrasco  Manuel Carrasco (born January 15, 1981) is a S...   
4             Melendi  His full name is Ramón Melendi Espina. Spanish...   
5           Lady Gaga  Stefani Joanne Angelina Germanotta, known prof...   
...               ...                                                ...   
1651       El Punto40   <a href="https://www.last.fm/music/El+Punto40...   
1677           Yandel  Yandel is half of the superstar reggaeton duo ...   
1680            Alejo   <a href="https://www.last.fm/music/Alejo">Rea...   
1692           Blessd  Talento y carisma hacen del cantante urbano an...   
1733      Ryan Castro  Ryan Castro es una de las nuevas caras

/var/folders/hw/nyy8ymjs45zfg18c105h7kjh0000gn/T/ipykernel_27019/1797292586.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_last_fm['Similars'].replace('', pd.NA, inplace=True)  # Reemplazar vacíos por NA


In [58]:
# Guardar el DataFrame en un archivo CSV de Last_fm:
df_last_fm.to_csv('lastfm_data.csv')